## <font color="#747a7f">7.5</font> Mapas coropléticos

Um [mapa coroplético](https://pt.wikipedia.org/wiki/Mapa_coropl%C3%A9tico) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas de símbolos dimensionados costumam ser mais precisos para ler, pois as pessoas tendem a ser melhores em estimar diferenças proporcionais entre a área de círculos do que entre tons de cores. No entanto, os mapas coropléticos são populares na prática e particularmente úteis quando muitos símbolos se tornam perceptivelmente esmagadores.

Por exemplo, embora os Estados Unidos tenham apenas 50 estados, eles têm milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado, no ano de recessão de 2008. Em alguns casos, os arquivos GeoJSON ou TopoJSON de entrada podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, entretanto, nós temos dois arquivos: nosso arquivo TopoJSON que inclui features de fronteira de condado (`usa`), e um arquivo de texto separado que contém estatísticas de desemprego:

In [ ]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

Para integrar nossas fontes de dados, nós precisaremos usar novamente a transformação de `lookup`, aumentando nossos dados de `geoshape` baseados em TopoJSON com taxas de desemprego. Podemos então criar um mapa que inclua uma codificação de `color` para o campo `rate` pesquisado.

In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True),
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nos estados das [Grandes Planícies](https://pt.wikipedia.org/wiki/Grandes_Plan%C3%ADcies) e Montanhas apresentam taxas baixas **e** altas. Esta variação é significativa ou é possivelmente um [artefato de tamanhos de amostra menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o domínio de escala superior (por exemplo, para ``0,2``) para ajustar o mapeamento de cores.*

Uma preocupação central para mapas coropléticos é a escolha de cores. Acima, nós usamos o esquema padrão `'yellowgreenblue'` do Altair para mapas de calor. Abaixo, nós comparamos outros esquemas, incluindo um esquema *sequencial de matiz único* (`teals`) que varia apenas em luminância, um esquema *sequencial de vários matizes* (`viridis`) que aumenta em luminância e matiz e um esquema *divergente* (`blueorange`) que usa um ponto médio branco:

In [ ]:
# função utilitária para gerar uma especificação de mapa para um esquema de cores oferecido
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

*Quais esquemas de cores você acha mais eficazes? Por que poderia ser? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito na documentação [Vega Color Schemes](https://vega.github.io/vega/docs/schemes/).*

## <font color="#747a7f">7.6</font> Projeções Cartográficas

Agora que nós temos alguma experiência na criação de mapas, vamos dar uma olhada mais de perto nas projeções cartográficas. Conforme explicado pela [Wikipedia](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cartogr%C3%A1fica),

> Todas as projeções de mapa necessariamente distorcem a superfície de alguma forma. Dependendo do propósito do mapa, algumas distorções são aceitáveis ​​e outras não; portanto, diferentes projeções de mapa existem para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades.



Algumas das propriedades que podemos querer considerar incluem:

* *Área:* A projeção distorce os tamanhos das regiões?

* *Orientação:* Uma linha reta corresponde a uma direção constante de viagem?

* *Distância:* Linhas de comprimento igual correspondem a distâncias iguais no globo?

* *Forma:* A projeção preserva relações espaciais (ângulos) entre pontos?

A seleção de uma projeção apropriada depende, portanto, do caso de uso do mapa. Por exemplo, se estivermos a avaliar o uso da terra e a extensão das questões fundiárias, poderemos escolher uma projecção de preservação da área. Se quisermos visualizar as ondas de choque que emanam de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir desse ponto. Ou, se quisermos ajudar a navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da *superfície de projeção*. As projeções cilíndricas, por exemplo, projetam pontos superficiais da esfera em um cilindro circundante; o cilindro “desenrolado” fornece então o nosso mapa. Como descreveremos abaixo, podemos alternativamente projetar na superfície de um cone (projeções cônicas) ou diretamente em um plano plano (projeções azimutais).

*Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! [Abra o caderno online Vega-Lite Cartographic Projections](https://observablehq.com/@vega/vega-lite-cartographic-projections). Use os controles naquela página para selecionar uma projeção e explorar parâmetros de projeção, como a ``escala`` (zoom) e a translação x/y (panning). Os controles de rotação ([guinada, inclinação, rolagem](https://pt.wikipedia.org/wiki/Eixos_do_avi%C3%A3o)) determinam a orientação do globo em relação à superfície que está sendo projetada.*

### <font color="#747a7f">7.6.1</font> Um tour por tipos específicos de projeção

**[Projeções cilíndricas](https://jjallaire.github.io/visualization-curriculum/altair_cartographic.html)** mapeiam a esfera em um cilindro circundante, então desenrolam o cilindro. Se o eixo principal do cilindro estiver orientado norte-sul, os meridianos são mapeados para linhas retas. [Projeções pseudocilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos “curvando-se” para longe do centro.

In [ ]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='Cilíndrica equidistante'),
    minimap.project(type='mercator').properties(title='Mercator'),
    minimap.project(type='transverseMercator').properties(title='Transversa de mercator'),
    minimap.project(type='naturalEarth1').properties(title='Natural da terra')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)



* [Cilíndrica equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cil%C3%ADndrica_equidistante) (``equirectangular``): Escala valores de coordenadas ``lat``, ``lon`` diretamente.

* [Mercator](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Mercator) (``mercator``): Projeta em um cilindro, usando ``lon`` diretamente, mas submetendo ``lat`` a uma transformação não linear. Linhas retas preservam orientações de bússola constantes ([linha de rumo](https://pt.wikipedia.org/wiki/Loxodromia)), tornando esta projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser bastante distorcidas.

* [Transversa de mercator](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_transversa_de_Mercator) (``transverseMercator``): Uma projeção mercator, mas com o cilindro delimitador girado para um eixo transversal. Enquanto a projeção Mercator padrão tem maior precisão ao longo do equador, a projeção Transversa de mercator é mais precisa ao longo do meridiano central.

* [Natural da terra](https://en.wikipedia.org/wiki/Natural_Earth_projection) (``naturalEarth1``): Uma projeção pseudocilíndrica projetada para mostrar toda a Terra em uma visão.


[Projeções cônicas](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera em um cone e, então, desenrolam o cone no plano. Projeções cônicas são configuradas por dois *paralelos padrões*, que determinam onde o cone intercepta o globo.

In [ ]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='Área cônica igual'),
    minimap.project(type='conicEquidistant').properties(title='Cônica equidistante'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='Cônica Conforme'),
    minimap.project(type='albers').properties(title='Albers'),
    minimap.project(type='albersUsa').properties(title='Albers EUA')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

* [Área cônica igual](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``conicEqualArea``): projeção cônica que preserva a área. A forma e a distância não são preservadas, mas são aproximadamente precisas dentro dos paralelos padrão.

* [Cônica equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_c%C3%B4nica_equidistante_meridiana) (``conicEquidistant``): projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.

* [Cônica Conforme](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_c%C3%B4nica_conforme_de_Lambert) (``conicConformal``): projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.

* [Albers](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``albers``): uma variante da projeção de área cônica igual com paralelos padrão otimizados para criar mapas dos Estados Unidos.

* [Albers EUA](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_de_Albers) (``albersUsa``): uma projeção híbrida para os 50 estados dos Estados Unidos da América. Esta projeção une três projeções de Albers com parâmetros diferentes para os EUA continentais, Alasca e Havaí.





[Projeções azimutais](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente em um plano.

In [ ]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='Área Azimutal Igual'),
    minimap.project(type='azimuthalEquidistant').properties(title='Azimutal Equidistante'),
    minimap.project(type='orthographic').properties(title='Ortográfico'),
    minimap.project(type='stereographic').properties(title='Estereográfico'),
    minimap.project(type='gnomonic').properties(title='Gnomônico')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)



*   [Área Azimutal Igual](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_azimutal_de_Lambert) (``azimuthalEqualArea``): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).

*   [Azimutal Equidistante](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_azimutal_equidistante) (``azimuthalEquidistant``): Preserva a distância proporcional do centro de projeção para todos os outros pontos do globo.

* [Ortográfico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_ortogr%C3%A1fica) (``orthographic``): Projeta um hemisfério visível em um plano distante. Corresponde aproximadamente a uma visão da Terra do espaço sideral.

* [Estereográfico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_estereogr%C3%A1fica) (``stereographic``): Preserva a forma, mas não a área ou a distância.

* [Gnomônico](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_gnom%C3%B4nica) (``gnomonic``): Projeta a superfície da esfera diretamente em um plano tangente. [Grandes círculos](https://pt.wikipedia.org/wiki/C%C3%ADrculo_m%C3%A1ximo) ao redor da Terra são projetados em linhas retas, mostrando o caminho mais curto entre os pontos.



## <font color="#747a7f">7.7</font> Coda: Dados Geográficos de Disputa

Todos os exemplos acima são extraídos da coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (aeroportos, taxas de desemprego). Um desafio comum para começar com a visualização geográfica é coletar os dados necessários para sua tarefa. Vários provedores de dados são abundantes, incluindo serviços como o [United States Geological Survey](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics) e o [U.S. Census Bureau](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).

Em muitos casos, você pode ter dados existentes com um componente geográfico, mas requer medidas ou geometria adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

1.   Visite o [Natural Earth Data](https://www.naturalearthdata.com/downloads/) e navegue para selecionar dados para regiões e resoluções de interesse. Baixe o(s) arquivo(s) zip correspondente(s).
2.   Vá para o [MapShaper](https://mapshaper.org/) e solte o arquivo zip baixado na página. Revise os dados conforme desejado e, em seguida, "Exportar" os arquivos TopoJSON ou GeoJSON gerados.
3.   Carregue os dados exportados do MapShaper para uso com o Altair!





Claro, muitas outras ferramentas - tanto de código aberto quanto proprietárias - existem para trabalhar com dados geográficos. Para mais sobre geo-data wrangling e criação de mapas, veja a série de tutoriais de Mike Bostock sobre [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## <font color="#747a7f">7.8</font> Sumário

Neste ponto, nós apenas mergulhamos os dedos dos pés nas águas da cartografia. (*Você não esperava que um único caderno transmitisse séculos de aprendizado, esperava?*) Por exemplo, deixamos intocados tópicos como [cartogramas](https://pt.wikipedia.org/wiki/Cartograma) e [topografia](https://pt.wikipedia.org/wiki/Topografia) de transmissão - como no livro esclarecedor de Imhof, [Cartographic Relief Presentation](https://books.google.com.br/books?id=cVy1Ms43fFYC&redir_esc=y). No entanto, agora você deve estar bem equipado para criar uma rica variedade de geovisualizações. Para mais, o livro de MacEachren [How Maps Work: Representation, Visualization, and Design](https://books.google.com.br/books?id=xhAvN3B0CkUC&redir_esc=y) fornece uma visão geral valiosa da cartografia da perspectiva da visualização de dados.